In [2]:
import os
import keyring
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import tiktoken

# Initialize the embeddings model
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
embeddings = OpenAIEmbeddings(model='text-embedding-3-large', api_key=OPENAI_API_KEY)

# Set up LangSmith observability
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = keyring.get_password('langsmith', 'learning_agent')
os.environ['LANGCHAIN_PROJECT'] = "pr-stupendous-hood-8"

In [3]:
### Step 1. Load and Process Local Document (PDF)
file_path = os.path.join(os.getcwd(), "docs", "faiss.pdf")
pdf_loader = PyPDFLoader(file_path)
pdf_documents = pdf_loader.load()

# Split PDF into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_pdf_documents = text_splitter.split_documents(pdf_documents)

In [5]:
### Step 2. Load and Process Web Content
# Define URLs to fetch web content
urls = [
    "https://github.com/facebookresearch/faiss",
    "https://github.com/facebookresearch/faiss/wiki"
]

# Load web content
web_loader = WebBaseLoader(web_paths=urls)
web_documents = web_loader.load()

# Split web documents into chunks
split_web_documents = text_splitter.split_documents(web_documents)

Created a chunk of size 3269, which is longer than the specified 1000
Created a chunk of size 1090, which is longer than the specified 1000
Created a chunk of size 4705, which is longer than the specified 1000
Created a chunk of size 1093, which is longer than the specified 1000


In [6]:
### Step 3: Combine Local and Web Documents

# Combine documents from both local and web sources
all_documents = split_pdf_documents + split_web_documents

### Step 4: Index documents in Chroma Vector Store

# Index all documents in Chroma
db = Chroma.from_documents(documents=all_documents, embedding=embeddings)
print("All documents indexed in Chroma successfully.")

All documents indexed in Chroma successfully.


In [7]:
## Step 5: Define a Retriever

# Define a retriever to fetch relevant documents from the combined sources
retriever = db.as_retriever(search_type='similarity', search_kwargs={"k":5})

### Step 6: Define the Prompt Template for the LLM
prompt = ChatPromptTemplate.from_template(
    """ 
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise.
    Question: {question}
    Context: {context}
    Answer:
    """
)

### Step 7: Initialize the ChatOpenAI Model
model = ChatOpenAI(model='gpt-4o-mini', temperature=0, api_key=OPENAI_API_KEY)

### Step 8: Set up the Retrieval-Augmented Generation(RAG) Chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

### Step 9. Ask a Question and Generate a Response
question = "Who are the main authors of Faiss?"
for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

The main authors of Faiss are Matthijs Douze, Alexandr Guzhva, Chengqi Deng, Jeff Johnson, Gergely Szilvasy, Pierre-Emmanuel Mazaré, Maria Lomeli, Lucas Hosseini, and Hervé Jégou. Hervé Jégou initiated the project and wrote its first implementation, while others contributed significantly to various components. The project is developed primarily at Meta AI Research.